In [27]:
!pip install -qU kaggle

^C


In [ ]:
!md "C:\Users\KIIT\.kaggle"
!icacls "C:\Users\KIIT\.kaggle\kaggle.json" /grant Everyone:F

A subdirectory or file C:\Users\KIIT\.kaggle already exists.


processed file: C:\Users\KIIT\.kaggle\kaggle.json
Successfully processed 1 files; Failed processing 0 files


In [ ]:
!kaggle datasets download ryanrudes/yttts-speech


  0%|          | 0.00/9.91G [00:00<?, ?B/s]
  0%|          | 1.00M/9.91G [00:01<4:27:48, 662kB/s]
  0%|          | 2.00M/9.91G [00:01<2:13:41, 1.33MB/s]
  0%|          | 3.00M/9.91G [00:01<1:23:01, 2.14MB/s]
  0%|          | 4.00M/9.91G [00:02<57:02, 3.11MB/s]  
  0%|          | 6.00M/9.91G [00:02<32:59, 5.37MB/s]
  0%|          | 8.00M/9.91G [00:02<23:02, 7.69MB/s]
  0%|          | 10.0M/9.91G [00:02<18:31, 9.57MB/s]
  0%|          | 12.0M/9.91G [00:02<16:02, 11.0MB/s]
  0%|          | 14.0M/9.91G [00:02<14:24, 12.3MB/s]
  0%|          | 16.0M/9.91G [00:02<14:39, 12.1MB/s]
  0%|          | 19.0M/9.91G [00:03<13:07, 13.5MB/s]
  0%|          | 21.0M/9.91G [00:03<12:00, 14.7MB/s]
  0%|          | 23.0M/9.91G [00:03<13:58, 12.7MB/s]
  0%|          | 25.0M/9.91G [00:03<12:37, 14.0MB/s]
  0%|          | 27.0M/9.91G [00:03<12:08, 14.6MB/s]
  0%|          | 29.0M/9.91G [00:03<11:28, 15.4MB/s]
  0%|          | 31.0M/9.91G [00:03<11:25, 15.5MB/s]
  0%|          | 34.0M/9.91G [00:04<11:20, 15.6

In [ ]:
# !zipfile yttts-speech.zip
!tar -xf yttts-speech.zip

In [1]:
!pip install -qU bs4 tqdm datasets

In [5]:
import os
import time
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [6]:
video_ids = os.listdir("data")
video_ids[:5]

['-9evrZnBorM', '-Y7PLaxXUrs', '0O1UXKh-Yck', '1DYmgoij4FQ', '1L83tM8nwHU']

In [7]:
splits = sorted(os.listdir(f"data/{video_ids[0]}"))
splits[:5]

['00_00_00,140-00_00_02,490',
 '00_00_02,490-00_00_05,069',
 '00_00_05,069-00_00_07,200',
 '00_00_07,200-00_00_10,889',
 '00_00_10,889-00_00_15,570']

In [8]:
with open(f"data/{video_ids[0]}/{splits[0]}/subtitles.txt") as f:
    text = f.read()
print(text)

hello everyone today we're looking at


In [7]:
documents = []
for video_id in tqdm(video_ids):
    splits = sorted(os.listdir(f"data/{video_id}"))
    # we start at 00:00:00
    start_timestamp = "00:00:00"
    passage = ""
    for i, s in enumerate(splits):
        with open(f"data/{video_id}/{s}/subtitles.txt") as f:
            # append tect to current chunk
            out = f.read()
            passage += " " + out
        # average sentence length is 75-100 characters so we will cut off
        # around 3-4 sentences
        if len(passage) > 360:
            # now we've hit the needed length create a record
            # extract the end timestamp from the filename
            end_timestamp = s.split("-")[1].split(",")[0]
            end_timestamp = end_timestamp.replace("_", ":")  # replace underscores with colons
            # extract string timestamps to actual datetime objects
            start = time.strptime(start_timestamp,"%H:%M:%S")
            end = time.strptime(end_timestamp,"%H:%M:%S")
            # now we extract the second/minute/hour values and convert
            # to total number of seconds
            start_second = start.tm_sec + start.tm_min*60 + start.tm_hour*3600
            end_second = end.tm_sec + end.tm_min*60 + end.tm_hour*3600
            # save this to the documents list
            documents.append({
                "video_id": video_id,
                "text": passage,
                "start_second": start_second,
                "end_second": end_second,
                "url": f"https://www.youtube.com/watch?v={video_id}&t={start_second}s",
            })
            # now we update the start_timestamp for the next chunk
            start_timestamp = end_timestamp
            # refresh passage
            passage = ""

100%|██████████| 127/127 [00:53<00:00,  2.38it/s]


In [ ]:
documents[:3]

[{'video_id': '-9evrZnBorM',
  'text': " hello everyone today we're looking at bird pre-training of deep bi-directional transformers for language understanding by Jacob Devlin and then why Chung Kenton Lee Cristina Tata Nova these are people from Google AI language so you're about to see the most hyped model currently so basically Bert is a model that takes as an input language sub token sequences and outputs various",
  'start_second': 0,
  'end_second': 34,
  'url': 'https://www.youtube.com/watch?v=-9evrZnBorM&t=0s'},
 {'video_id': '-9evrZnBorM',
  'text': " things so it can it can be made to do various things almost any NLP tasks with basically little training because the Bert model comes pre trained on a very large corpus and we're gonna see how that's done all right so the paper introduces basically the kind of current state of the art of language models and they say okay what they want to do new is they want to do",
  'start_second': 34,
  'end_second': 66,
  'url': 'https://www.

In [ ]:
!pip install lxml

In [ ]:
import lxml  # if on mac, pip/conda install lxml

metadata = {}
for _id in tqdm(video_ids):
    r = requests.get(f"https://www.youtube.com/watch?v={_id}")
    soup = BeautifulSoup(r.content, 'html.parser')  # lxml package is used here
    try:
        title = soup.find("meta", property="og:title").get("content")
        thumbnail = soup.find("meta", property="og:image").get("content")
        metadata[_id] = {"title": title, "thumbnail": thumbnail}
    except Exception as e:
        print(e)
        print(_id)
        metadata[_id] = {"title": "", "thumbnail": ""}

len(metadata)

 24%|██▎       | 30/127 [00:22<01:00,  1.60it/s]

'NoneType' object has no attribute 'get'
bM9BXu9lPZ0


 33%|███▎      | 42/127 [00:32<01:01,  1.39it/s]

'NoneType' object has no attribute 'get'
fpDaQxG5w4o


100%|██████████| 127/127 [01:34<00:00,  1.34it/s]


127

In [ ]:
metadata['ZPewmEu7644']

{'title': 'GANS for Semi-Supervised Learning in Keras (7.4)',
 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}

In [ ]:
for i, doc in enumerate(documents):
    _id = doc['video_id']
    meta = metadata[_id]
    # add metadata to existing doc
    documents[i] = {**doc, **meta}

In [ ]:
documents[0]

{'video_id': '-9evrZnBorM',
 'text': " hello everyone today we're looking at bird pre-training of deep bi-directional transformers for language understanding by Jacob Devlin and then why Chung Kenton Lee Cristina Tata Nova these are people from Google AI language so you're about to see the most hyped model currently so basically Bert is a model that takes as an input language sub token sequences and outputs various",
 'start_second': 0,
 'end_second': 34,
 'url': 'https://www.youtube.com/watch?v=-9evrZnBorM&t=0s',
 'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding',
 'thumbnail': 'https://i.ytimg.com/vi/-9evrZnBorM/maxresdefault.jpg'}

In [ ]:
import json

with open("train.jsonl", "w") as f:
    for doc in documents:
        json.dump(doc, f)
        f.write('\n')

In [9]:
with open("train.jsonl") as f:
    d = f.readlines()

In [10]:
d[:3]

['{"video_id": "-9evrZnBorM", "text": " hello everyone today we\'re looking at bird pre-training of deep bi-directional transformers for language understanding by Jacob Devlin and then why Chung Kenton Lee Cristina Tata Nova these are people from Google AI language so you\'re about to see the most hyped model currently so basically Bert is a model that takes as an input language sub token sequences and outputs various", "start_second": 0, "end_second": 34, "url": "https://www.youtube.com/watch?v=-9evrZnBorM&t=0s", "title": "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding", "thumbnail": "https://i.ytimg.com/vi/-9evrZnBorM/maxresdefault.jpg"}\n',
 '{"video_id": "-9evrZnBorM", "text": " things so it can it can be made to do various things almost any NLP tasks with basically little training because the Bert model comes pre trained on a very large corpus and we\'re gonna see how that\'s done all right so the paper introduces basically the kind of current sta

In [11]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

In [12]:
from datasets import load_dataset

yt = load_dataset(
    'pinecone/yt-transcriptions',
    split = 'train',
    revision = '926a45'
)

Found cached dataset json (C:/Users/KIIT/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-c0a57d2f8951521b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [13]:
for x in yt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [14]:
from sentence_transformers import SentenceTransformer
import torch

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [15]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

In [16]:
text = []
for i in yt:
  text.append(i['text'])
text[0]

" hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you"

In [17]:
from tqdm.auto import tqdm
import numpy as np
import faiss

# Assume we have a dimension for the embeddings based on first batch
dim = 768
batch_size = 74

# Setup FAISS index
nlist = 20  # number of Voronoi cells (i.e., clusters)
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, nlist)
assert not index.is_trained

# We'll keep a separate array for metadata that aligns with our FAISS ids
meta = []

# Prepare IDs for FAISS (needs to be a consecutive list of integers)
next_id = 0

for i in tqdm(range(0,len(yt),batch_size)):
    i_end = min(i+batch_size,len(yt))
    batch = yt[i:i_end]                          # extract batch from YT transactions
    embeds = retriever.encode(batch['text']).tolist()   # encode a batch

    if not index.is_trained:
        # If index is not trained yet, train it with the first batch
        index.train(np.array(embeds))
        assert index.is_trained

    ids = np.array(range(next_id, next_id + len(embeds))) # create IDs for this batch
    index.add_with_ids(np.array(embeds), ids)  # add this batch to FAISS index

    meta += [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]

    next_id += len(embeds)  # update the next ID for the next batch

100%|██████████| 153/153 [37:45<00:00, 14.80s/it]


In [18]:
query = 'what is principal component analysis?'
xq = retriever.encode([query])
xq = np.array(xq).reshape(-1, 1)
print("Shape of xq:", np.array(xq).shape)


Shape of xq: (768, 1)


In [19]:
query = 'what is principal component analysis?'
xq = retriever.encode([query]).tolist()

k = 3 # number of nearest neighbors
D, I = index.search(np.array(xq), k)

# D and I are distances and index lists respectively
for i, d in zip(I[0], D[0]):
    # Here 'i' is the id returned from faiss, we use it to retrieve the corresponding metadata
    print(f"ID: {i}, Distance: {d}")
    print("Title:", meta[i]['title'])
    print("Text:", meta[i]['text'])
    print("URL:", meta[i]['url'])
    print("Thumbnail:", meta[i]['thumbnail'], end='\n\n')

ID: 654, Distance: 0.9018356800079346
Title: Concept Learning with Energy-Based Models (Paper Explained)
Text:  is something red concepts in the same places so this is a PCA and then blue I think these blue is the attention codes for identify the red things and in red or the generation code for make something red and they will be put in the same place which is pretty cool it means that the energy function really learns the feature of something being red I find this pretty pretty neat and then
URL: https://www.youtube.com/watch?v=Cs_j-oNwGgg&t=1933s
Thumbnail: https://i.ytimg.com/vi/Cs_j-oNwGgg/maxresdefault.jpg

ID: 3782, Distance: 0.9956833124160767
Title: Yann LeCun: "Energy-Based Self-Supervised Learning"
Text:  in PCA the the the region of the space that is perfectly reconstructed is the principal subspace right in this case here if the data points are sampled from this from the sparrow the principal subspace here dimension 1 is this so this has become our reconstruction error 0 an

In [20]:
import pickle
# Save FAISS index to disk
faiss.write_index(index, "faiss.index")

# Save metadata to disk
with open("meta.pkl", "wb") as f:
    pickle.dump(meta, f)